# Universes

A universe is a collection of cells that can be used as a repeatable unit in the geometry. At a minimum, there must be one "root" universe, which gets passed to `openmc.Geometry(root)`. But you can also use universes to repeat a collection of cells multiple times throughout a geometry. Here, we will explore some basic features of universes.

We'll start by making a universe which looks similar to the pincell we built earlier - say, a square of UO2 enclosed by an infinite region of water. First, we set up our materials and create our geometry.

Let's plot our universe to confirm we know what we've built. Note that because the `region` for `outside_cell` is simply all of the space not inside the pin, the water extends to infinity.

Now let's suppose that I want to fill this universe into an enclosing cell, a cylinder of radius 5 cm. Let's first create this cylinder, and then we will fill it with our universe.

We can see that our `big_cell`, the large cylinder, has been filled with the universe we declared earlier. Let's increase the complexity a bit to understand how this filling works. What if the filled universe is not located at the origin, but is instead shifted to a different position?

We see that when we fill a universe inside of another cell, that there's (by default) no transformation of coordinates. It's as if we are directly underlaying the filled universe into a "higher-up" cell. You can shift the position of the universe filling a cell with the `Cell.translation` attribute. 

There are similar adjustments you can make, like rotations. The `Cell.rotation` method lists the rotation angles (in degrees) for rotations about the x, y, and z axes.

# Lattices

Lattices are a convenient way to (i) repeat a universe multiple times in space, while (ii) automatically translating that universe's origin to different positions in space. 

In this section, we will build one of the assemblies from the BEAVRS benchmark. This is a Pressurized Water Reactor (PWR) assembly with fuel pins, guide tubes, and borosilicate glass burnable poisons. A diagram of the assembly is plotted below.

<img src="assembly_diagram.png" alt="drawing" width="350"/>

In order to build this geometry, we will need to define four universes -- one for a fuel pin (the lilac square with nothing indicated in them), one for a poison pin (B), one for an instrumentation pin (I), and one for a guide tube (G). 

Before we can discuss lattices, we need to build the materials, cells, and universes for each of these.

### Fuel pin universe

To build our fuel pin universe, we will require three materials. Note that we treat the helium gap as vacuum by setting `fill=None`.

In [ ]:
uo2 = openmc.Material(name='uo2')
uo2.add_element('U', 1.0, enrichment=3.0)
uo2.add_nuclide('O16', 2.0)
uo2.set_density('g/cm3', 10.0)

zirconium = openmc.Material(name='zirconium')
zirconium.add_element('Zr', 1.0)
zirconium.set_density('g/cm3', 6.6)

water = openmc.Material(name='water')
water.add_nuclide('H1', 2)
water.add_nuclide('O16', 1)
water.set_density('g/cm3', 0.7)
water.add_s_alpha_beta('c_H_in_H2O')

When building a complex geometry, it is helpful to plot each universe as you go along. Let's plot this pincell now. 

By default, OpenMC will select a random color for each material when plotting; instead, we can specify some custom values if we like.

In [ ]:
colors = {
    water: 'lightblue',
    zirconium: 'gray',
    uo2: 'red'
}

### Guide tube universe
A guide tube is an annulus of zirconium within which water flows. These tubes are used to receive control rods.

### Pyrex burnable poison universe
The burnable absorber universe is a series of annular cylinders enclosing an annular pyrex layer. The geometry is defined as follows:

- R < 0.21 cm, void
- 0.21 cm < R < 0.23 cm, zirconium
- 0.23 cm < R < 0.24 cm, void
- 0.24 cm < R < 0.43 cm, pyrex
- 0.43 cm < R < 0.44 cm, void
- 0.44 cm < R < 0.48 cm, zirconium
- 0.48 cm < R < 0.56 cm, water
- 0.56 cm < R < 0.60 cm, zirconium
- 0.60 cm < R, water

In [ ]:
pyrex = openmc.Material(name='pyrex')
pyrex.add_element('B', 0.49)
pyrex.add_element('O', 4.7)
pyrex.add_element('Al', 0.17)
pyrex.add_element('Si', 1.8)
pyrex.set_density('g/cm3', 2.26)

In [ ]:
colors[pyrex] = 'green'

To create the geometry, we're going to use some advanced features. First, we'll use a [list comprehension](https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions), which is a way of creating a list in Python that embeds a for loop.

To create this pin, we're going to use a function provided by OpenMC specifically for this purpose, [`openmc.model.pin`](https://docs.openmc.org/en/stable/pythonapi/model.html).

## Lattices in OpenMC

OpenMC has `RectLattice` and `HexLattice` objects, to place universes within a rectangular or hexagonal lattice, respectively. For our fuel assembly, we need to use RectLattice. 

When creating a rectangular lattice, we need to define:

- The lower-left coordinates of the lattice (`.lower_left`)
- The size of each lattice element (`.pitch`)
- The 2D arrangement of universes (`.universes`)
- (optionally) A universe that is used outside of the defined region (`.outer`)

By default, the extent of a lattice only covers the actual lattice "slots." If we try to plot our geometry but with a wider width (so that we are including regions of space outside our 2x2 lattice), OpenMC will throw an error.

If we ever want to plot or have neutrons track through regions outside the lattice, but which are not also "chopped off" by some containing cell, we can specify the `.outer` parameter for the lattice. As an example, let's set the outer universe to an infinite universe of water.

## What exactly does `outer` mean?

To get a better sense of what the outer universe does, let's change the outer universe to the burnable poision universe:

# The BEAVRS Assembly

<img src="assembly_diagram.png" alt="drawing" width="350"/>

To make things a little easier, we'll create lists of (row, column) positions for the guide tubes and burnable poison rods:

In [ ]:
guide_tube_positions = [
    (2, 5), (2, 8), (2, 11),
    (5, 2), (5, 5), (5, 8), (5, 11), (5, 14),
    (8, 2), (8, 5), (8, 8), (8, 11), (8, 14),
    (11, 2), (11, 5), (11, 8), (11, 11), (11, 14),
    (14, 5), (14, 8), (14, 11)
]

burn_positions = [(3, 3), (3, 13), (13, 3), (13, 13)] 

Now we just have to add the boundary conditions and root universe to finish the geometry. Because each of our cells we used in constructing the lattice uses `transmission` boundary conditions (the default), we need to place the lattice inside some other containing cell, which has boundary conditions.

To run this input file, now all we need to do is specify the initial starting source and some run settings.

## Hexagonal Lattices
OpenMC also allows you to define hexagonal lattices. They are a little trickier, but as we'll see there are some helper methods that demystify how to assign universes.

We need to set the center of the lattice, the pitch, an outer universe (which is applied to all lattice elements outside of those that are defined), and a list of universes. Let's start with the easy ones first. Note that for a 2D lattice, we only need to specify a single number for the pitch.

Now we need to set the universes property on our lattice. It needs to be set to a list of lists of Universes, where each list of Universes corresponds to a ring of the lattice. The rings are ordered from outermost to innermost, and within each ring the indexing starts at the "top". To help visualize the proper indices, we can use the `show_indices()` helper method.

Let's set up a lattice where the first element in each ring is the guide tube universe and all other elements are regular fuel pin universes. From the diagram above, we see that the outer ring has 12 elements, the middle ring has 6, and the innermost degenerate ring has a single element.

Now let's put our lattice inside a circular cell.

Now let's say we want our hexagonal lattice orientated such that flat sides are parallel to the y-axis instead of the x-axis. This can be achieved by changing the orientation of the lattice from `y` to `x`: